original code found on https://pypi.org/project/drawSvg/

new code specific for our usecase

In [4]:
import drawSvg as draw
from drawSvg.widgets import DrawingWidget
import sys
sys.path.insert(0,"..")
from hyperbolic.poincare.shapes import *
from constructions import *
from Triangle import *
from deltaNeigbourhood import *

backgroundStyle = dict(stroke='#cc6500', stroke_width=0.01, fill='#ff7f00')
polyStyle1 = dict(stroke='#793e82', stroke_width=0.005,  fill='#377eb8', fill_opacity=1)
polyStyle2 = dict(stroke='#e94749', stroke_width=0.005, fill='#e41a1c', fill_opacity=0.75)
lineStyle1 = dict(hwidth=0.04, fill='#984ea3')
pointStyle1 = dict(hradius=0.02, fill='#984ea3')
lineStyle2 = dict(hwidth=0.04, fill='#4daf4a')
pointStyle2 = dict(hradius=0.02, fill='#4daf4a')

delta=0.3
PList=[]

clicks=0
approximated=False

# Create drawing
d = draw.Drawing(2.1, 2.5, origin='center')
d.setRenderSize(500)
d.append(draw.Circle(0, 0, 1, **backgroundStyle))
g=draw.Group()
d.append(g)

def drawText(text1, text2):
    g.draw(draw.Text(text1, 0.1, 0, 1.1, center=0.7))
    g.draw(draw.Text(text2, 0.1, 0, 1.2, center=0.7))
    

def drawLines(points):
    for i,p1 in enumerate(points):
        for p2 in points[i+1:]:
            g.draw(Line.fromPoints(*p1, *p2, segment=True), **lineStyle1)
    for p1 in points:
        g.draw(p1, **pointStyle2)

def drawTriAndNbh(points, edgeNum):
    global approxdelta
    Tri=Triangle.fromVertices(points)
    if clicks==3:
        approxdelta = Tri.approx()
    if approximated:
        offset = approxdelta
    else:
        offset = delta
    g.draw(Tri,**polyStyle1)
    g.draw(deltaNbh(Tri, offset, edgeNum), **polyStyle2)

text='click inside the circle to add a point'
deltatext='delta is set to = {}'.format(delta)
    
        
widget = DrawingWidget(d)
@widget.mousedown

def mousedown(widget, x, y, info):
    global clicks, approximated
    if (x**2 + y**2) ** 0.5 + 2e-10 < 1:
        clicks+=1
        if clicks<3:
            PList.append(Point(x, y))
            g.children.clear()
            drawLines(PList)
            drawText(text, deltatext)
        elif clicks==3:
            PList.append(Point(x, y))
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)
        else:
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)       
    else:
        approximated = not approximated
        if approximated:
            text='click to change to set delta'
            deltatext='approximated delta = {}'.format(approxdelta)
        else:
            text='click to change to approximated delta'
            deltatext='set delta = {}'.format(delta)
        if len(PList)==3:
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)
        else:
            g.children.clear()
            drawLines(PList)
            drawText(text, deltatext)

    widget.refresh()
        
        
@widget.mousemove
def mousemove(widget, x, y, info):
    global text, deltatext
    if (x**2 + y**2) ** 0.5 + 2e-10 < 1:
        if len(PList)<3:
            text='click to add point'
            g.children.clear()
            g.draw(Point(x, y),**pointStyle2)
            drawLines([Point(x, y)] + PList)
            drawText(text, deltatext)
        else:
            text='click to change edge of deltaneigbourhood'
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)    
    else:
        if len(PList)==3:
            if approximated:
                text='click to change to set delta'
                deltatext='approximated delta = {}'.format(approxdelta)
            else:
                text='click to change to approximated delta'
                deltatext='set delta = {}'.format(delta)
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)
        else:
            text='click inside the circle to add a point'
            g.children.clear()
            drawLines(PList)
            drawText(text, deltatext)

    widget.refresh()

widget


DrawingWidget()